# Third Party Conversions

This notebook looks into some different approaches we could take in allowing third party conversions in `extendr`: [#440](https://github.com/extendr/extendr/issues/440). This is related to the question of whether we use `TryFrom` or a custom trait ([#498](https://github.com/extendr/extendr/issues/498)), but only indirectly.

The main factors we need to consider are whether each approach allows users to implement an extendr conversion for:
* Their own types. We will use `MyStruct` as a proxy for this. This is the easiest to achieve
* Types that already have a conversion in `extendr` (the user wants to override this conversion). We will use the `bool` type as an example of this.
* Types that neither the user nor `extendr` owns. We will use `String` as an example of this, since it is owned by `std`.

Aside: we define a `cat_rust` function for displaying source code that isn't executed directly in this notebook:

In [5]:
:dep bat
use bat::PrettyPrinter;

fn cat_rust(path: &str){
    bat::PrettyPrinter::new()
        .input_file(path)
        .language("rust")
        .theme("GitHub")
        .print()
        .unwrap();
}

# Current Behaviour

We use the `current` crate to simulate what it's like to import `extendr-api` using the current behaviour.

In [6]:
:dep current = {path = "./current"}

`current` includes simple analogues of the real traits we would use in `extendr-api`. We also have an example conversion implementation for `bool`, which simulates the way in which `extendr-api` provides built-in conversions: 

In [8]:
cat_rust("current/src/lib.rs");

pub struct Robj;
pub trait FromRobj: Sized {
    fn from_robj(robj: &Robj) -> Result<Self, &'static str>;
}
impl FromRobj for bool {
    fn from_robj(_robj: &Robj) -> Result<Self, &'static str> {
        Ok(true)
    }
}


Next, let's pretend this is the struct we are converting from R.

In [9]:
struct MyStruct;

In [10]:
impl current::FromRobj for MyStruct {
    fn from_robj(_: &current::Robj) -> Result<Self, &'static str> {
        Ok(MyStruct)
    }
}

This works, so the first criteria is satisfied ✅

For the second criteria, overriding an existing conversion, we get an Orphan Rule violation ❎:

In [13]:
impl current::FromRobj for bool {
    fn from_robj(_robj: &current::Robj) -> Result<Self, &'static str> {
        Ok(false)
    }
}

Error: only traits defined in the current crate can be implemented for primitive types

Likewise for the third criteria, when we try to implement a conversion for a type that we don't own. In this case, `String` is part of the `std` crate so is not under our ownership:

In [14]:
impl current::FromRobj for String {
    fn from_robj(_: &current::Robj) -> Result<Self, &'static str> {
        Ok(String::new())
    }
}

Error: only traits defined in the current crate can be implemented for types defined outside of the crate

# Newtype

Newtype is the current workaround to the above issues. Indeed it's the standard workaround to the Orphan Rule in Rust generally.

In [15]:
struct MyString(String);

impl current::FromRobj for MyString {
    fn from_robj(_: &current::Robj) -> Result<Self, &'static str> {
        Ok(MyString(String::from("foo")))
    }
}

We are able to successfully create a conversion for a third party type.

However, this adds some overhead to the user. If we try to use an extendr function with the original type (`String`), it will fail:

In [16]:
// Imagine this has the #[extendr] attribute
fn print_string(x: String){
    println!("{}", x)
}

// And imagine this is being called from R inside of extendr
print_string(current::FromRobj::from_robj(&current::Robj).unwrap());

Error: the trait bound `String: FromRobj` is not satisfied

But as long as we make the function accept the newtype it will be fine:

In [17]:
// Imagine this has #[extendr]
fn print_string(x: MyString){
    // Unwrap the newtype
    let unwrapped: String = x.0;
    
    // We can now use the String as normal 
    println!("{}", unwrapped)
}
print_string(current::FromRobj::from_robj(&current::Robj).unwrap());

foo


This approach also works for overriding an existing conversion:

In [18]:
struct MyBool(bool);
impl current::FromRobj for MyBool {
    fn from_robj(_robj: &current::Robj) -> Result<Self, &'static str> {
        Ok(MyBool(false))
    }
}

# Generic Conversion Trait

Another option, [as suggested here](https://github.com/extendr/extendr/issues/440#issuecomment-1448873888), is to define the `FromRobj` trait as generic over a marker trait:

In [7]:
:dep generic_trait = {path = "generic_trait"}

Firstly note that, since this approach requires the trait to be generic, we **could not** do this when using `TryFrom` as the conversion trait. This would only work with a `FromRobj` type custom trait:

In [20]:
cat_rust("generic_trait/src/lib.rs");

pub struct Robj;
pub trait FromRobj<Marker>: Sized {
    fn from_robj(robj: &Robj) -> Result<Self, &'static str>;
}
pub struct CoreConversion;
impl FromRobj<CoreConversion> for bool {
    fn from_robj(_robj: &Robj) -> Result<Self, &'static str> {
        return Ok(true);
    }
}


Firstly note that we can now define a third party conversion without encountering the orphan rule:

In [21]:
struct ThirdPartyMarker;
impl generic_trait::FromRobj<ThirdPartyMarker> for String {
    fn from_robj(_robj: &generic_trait::Robj) -> Result<Self, &'static str> {
        return Ok(String::from("foo"));
    }
}

In addition, the marker trait is inferred automatically (note the `FromRobj` rather than `FromRobj<ThirdPartyMarker>`):

In [22]:
let x: String = generic_trait::FromRobj::from_robj(&generic_trait::Robj).unwrap();
x

"foo"

However, let's say that we try to redefine an existing conversion:

In [23]:
impl generic_trait::FromRobj<ThirdPartyMarker> for bool {
    fn from_robj(_robj: &generic_trait::Robj) -> Result<Self, &'static str> {
        return Ok(false);
    }
}
let x: bool = generic_trait::FromRobj::from_robj(&generic_trait::Robj).unwrap();
x

Error: cannot call associated function on trait without specifying the corresponding `impl` type

Rust can't automatically infer the marker struct to use, and so this fails to compile.

This would break the conversion within extendr's internals, and it would do so with a confusing warning message.

# Custom Conversion Trait

It might seem that we could define a custom trait for each third party trait, and then simply tell extendr to use it instead of `FromRobj`. One might envisage this looking like:

```rust
#[extendr(from_trait = MyFromRobj)]
```

We can try to "copy" the existing implementations on `FromRobj` using a blanket implementation on `MyFromRobj`:

In [24]:
trait MyFromRobj : Sized {
    fn from_robj(_: &current::Robj) -> Result<Self, &'static str>;
}
impl<T> MyFromRobj for T where T : current::FromRobj {
    fn from_robj(robj: &current::Robj) -> Result<Self, &'static str> {
        current::FromRobj::from_robj(robj)
    }
}

However we get a compiler error if we try to re-implement a conversion:

In [25]:
impl MyFromRobj for bool {
    fn from_robj(robj: &current::Robj) -> Result<Self, &'static str> {
        Ok(true)
    }
}

Error: conflicting implementations of trait `MyFromRobj` for type `bool`

# Summary

In summary, the two most promising approaches seem to be the newtype and the generic conversion approach. Each has different downsides and upsides.

Conversion Appoach | Own Types<sup>1</sup> | Third Party Types<sup>2</sup> | Overriding<sup>3</sup> | Unchanged Function<sup>4</sup> | Supports `TryFrom` <sup>5</sup>
--- | --- | --- | --- | --- | ---
Nothing | ✅ | ❌ | ❌ | ✅ | ✅
NewType | ✅ | ✅ | ✅ | ❌ | ✅
Generic Trait | ✅ | ✅ | ❌ | ✅ | ❌
Custom Trait |  ✅ | ✅ | ❓<sup>6</sup> | ❌ | ✅

<sup>1</sup> Can a user who is working on a crate other than `extendr` create conversions to and from `Robj` for their own types?

<sup>2</sup> Can a user who is working on a crate other than `extendr` create conversions to and from a third party type, that lives neither in their own crate, nor in extendr

<sup>3</sup> Can a user override the built-in extendr conversion if they want?

<sup>4</sup> Will the user have to modify the `#[extendr]` function in some way to support this style of conversion?

<sup>5</sup> Would this approach work if we kept the use of `TryFrom` as our conversion trait?

<sup>6</sup> They can technically do so, but then they lose the blanket implementation for normal `FromRobj` types which renders it fairly useless